In [5]:
import qrcode
from PIL import Image
import string    
import random
import pdfkit
import os
import glob
import datetime
from datetime import datetime
import requests
import pandas as pd
from pandas.io.json import json_normalize
import json
import requests.packages.urllib3
requests.packages.urllib3.disable_warnings()

In [6]:
# load wkhtmltopdf
base_path = os.path.join('./')#####只要改這邊
Download_PATH = os.path.join(base_path,'wkhtmltopdf','bin','wkhtmltopdf.exe')
config = pdfkit.configuration(wkhtmltopdf=Download_PATH)

In [7]:
def generate_qrcode(inp):
    url = "https://iot.thinktron.co/platform/ntpcagr/PDF/"+str(inp)
    qr = qrcode.QRCode(version=1,error_correction=qrcode.constants.ERROR_CORRECT_H,box_size=2,border=3)
    qr.add_data(url)
    qr.make(fit=True)
    img = qr.make_image(fill_color="black", back_color="white").convert('RGB')
    img.save(os.path.join(base_path,'PDF',str(inp)+".jpg"))

In [8]:
def id_generator(size=50, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))

In [54]:
!start .

In [16]:
def wkhtmltopdf(date,QR,region,section,num,MS,CA,NP,RA,CF,LS,DF):    
    head = '''
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" integrity="sha384-JcKb8q3iqJ61gNV9KGb8thSsNjpSL0n8PARn9HuZOnIxN0hoP+VmmDGMN5t9UJ0Z" crossorigin="anonymous">
    <link rel="stylesheet" href="https://use.fontawesome.com/releases/v5.8.2/css/all.css" integrity="sha384-oS3vJWv+0UjzBfQzYUhtDYW+Pj2yciDJxpsK1OYPAYjqT085Qq/1cq5FLXAZQ7Ay" crossorigin="anonymous">
    <link href="https://cdn.datatables.net/1.10.22/css/jquery.dataTables.min.css" rel="stylesheet" media="all">
    <link href="https://fonts.googleapis.com/css?family=Montserrat:400,700,200" rel="stylesheet" />
    <link href="https://maxcdn.bootstrapcdn.com/font-awesome/latest/css/font-awesome.min.css" rel="stylesheet">
    '''
    style = '''
    <style type="text/css">
        .waterMark {
            text-align:center;
            position:absolute;
            width:100%;
            height:100%;
            z-index:-9999;
            pointer-event:none;
            font-size:5em;
            font-weight:bold;
            font-family:Consolas;
            color:#EDEDED
        }
        .waterMark-inner{
            transform:rotate(-20deg);
            line-height:200px
        }
    </style>
    '''
    body = '''
    <div class="waterMark">
        <img style="position: fixed;bottom: 45%;left:28%;height:150; width:auto;" src="https://iot.thinktron.co/platform/ntpcagr/img/LOGO0.jpg" alt="NTPC" title="NTPC">
    </div>
    <p style="text-align:center;font-size:45px;font-family:Microsoft YaHei;">新北市政府農業局山坡地資訊查詢結果</p>
        <table id="searchResult-table" class="table table-sm table-bordered text-center" style="cursor: pointer;">
            <thead class="thead-dark">
                <tbody>
                <tr>
                    <th scope="col" style="width: 12.5%; vertical-align: middle;" >查詢時間：</th>
                    <th scope="col" colspan="4" style="width: 50%; vertical-align: middle !important;">{}</th>
                    <th scope="col" style="width: 12.5%; vertical-align: middle;" >查驗網址：</th>
                    <th scope="col" colspan="3";style="width: 37.5%; vertical-align: middle;"><img src="https://iot.thinktron.co/platform/ntpcagr/PDF/{}.jpg" alt="qr" title="qr"></th>
                </tr>
                <tr>
                    <th scope="col" style="width: 12.5%; vertical-align: middle;" >查詢地號</th>
                    <th scope="col" style="width: 12.5%; vertical-align: middle;" >是否座落<br>法定山坡地</th>
                    <th scope="col" style="width: 12.5%; vertical-align: middle;" >是否座落<br>特定水保區</th>
                    <th scope="col" style="width: 12.5%; vertical-align: middle;" >是否座落<br>國家公園</th>
                    <th scope="col" style="width: 12.5%; vertical-align: middle;" >是否座落<br>水庫集水區</th>
                    <th scope="col" style="width: 12.5%; vertical-align: middle;" >是否座落<br>保安林</th>
                    <th scope="col" style="width: 12.5%; vertical-align: middle;" >是否座落<br>山崩地滑<br>地質敏感區</th>
                    <th scope="col" style="width: 12.5%; vertical-align: middle;" >是否座落<br>土石流潛勢溪流<br>影響範圍</th>
                </tr>
                <tr>
                    <th scope="col" style="vertical-align: middle;">{}{}<br>{}號</th>
                    <th scope="col" style="vertical-align: middle;">{}</th>
                    <th scope="col" style="vertical-align: middle;">{}</th>
                    <th scope="col" style="vertical-align: middle;">{}</th>
                    <th scope="col" style="vertical-align: middle;">{}</th>
                    <th scope="col" style="vertical-align: middle;">{}</th>
                    <th scope="col" style="vertical-align: middle;">{}</th>
                    <th scope="col" style="vertical-align: middle;">{}</th>
                </tr>
                <tr>
                    <th scope="col" colspan="8"; style="text-align: left;">
                        <h5>
                        注意事項：<br>
                        1. 山坡地、特水區皆可作為本局正式證明文件之依據。<br>
                        2. 本頁查詢結果資料將保存一個月。<br>
                        3. 地籍資料仍以地政機關之資料為準，本系統查無相關地籍資料時，請逕洽本局查詢。<br>
                        4. 本系統之山坡地範圍不包含國有林事業區、試驗用林地及保安林地。<br>
                        5. 山坡地範圍及特定水土保持區倘因範圍有疑義者，請逕洽本局查詢。<br>
                        6. 國家公園、水庫集水區、保安林、地質敏感區及土石流潛勢溪流影響範圍由相關主管機關提供，不具證明效力。
                        </h5>
                    </th>
                </tr>
                </tbody>
            </thead>
        </table>
            '''.format(date,QR,region,section,num,MS,CA,NP,RA,CF,LS,DF)
    options = {
        'encoding': 'utf-8',  #utf-8
        'page-height': '210',
        'page-width': '297',
        'background': ''
    }
    
    f = open(os.path.join(base_path,'PDF',QR+".html"),'w')
    f.write(head)
    f.write(style)
    f.write(body)
    
    f.close()
    pdfkit.from_file(os.path.join(base_path,'PDF',QR+".html"), os.path.join(base_path,'PDF',QR+".pdf"), configuration=config, options=options)

In [17]:
def convert01(inp):
    if inp == 1 or str(inp) == "1":
        result = "是"
    else:
        result = "否"
    return result

In [18]:
def convert01MS(inp):
    if inp == 1 or str(inp) == "1":
        result = "是"
    elif inp == 3 or str(inp) == "3":
        result = "否"
    else:
        result = "請洽農業局"
    return result

In [19]:
def get_single_cad(AA48,AA49):
    cad_data = requests.get("https://iot.thinktron.co/ntpcagr-api/get_cad/"+str(AA48)+str(AA49),verify = False)
    cad_info = requests.get("https://iot.thinktron.co/ntpcagr-api/get_sectionInfo/"+str(AA48),verify = False)
    cad_data_json = json.loads(cad_data.text)
    cad_info_json = json.loads(cad_info.text)
    data = cad_data_json[0]['json_build_object']['features'][0]['properties']
    return [cad_info_json[0]['CODE_C'],cad_info_json[0]['NAME'],str(AA49)[0:4]+"-"+str(AA49)[4:8],convert01MS(data["MS"]),convert01(data["CA"]),convert01(data["NP"]),convert01(data["RA"]),convert01(data["CF"]),convert01(data["LS"]),convert01(data["DF"])]

In [20]:
def export_single_cad(AA48,AA49):    
    this_time = datetime.now().strftime("%Y/%m/%d %H:%M")
    qrstring = id_generator()
    generate_qrcode(qrstring)
    cad = get_single_cad(AA48,AA49)
    wkhtmltopdf(this_time,qrstring,cad[0],cad[1],cad[2],cad[3],cad[4],cad[5],cad[6],cad[7],cad[8],cad[9])

In [31]:
tmp_table = [get_single_cad("1822","00260000"), get_single_cad("1822","00270000")]

In [32]:
tmp_table

[['永和區', '福和段', '0026-0000', '否', '否', '否', '否', '否', '否', '否'],
 ['永和區', '福和段', '0027-0000', '否', '否', '否', '否', '否', '否', '否']]

In [21]:
export_single_cad("1822","00260000")

Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                      


'永和區福和段0026-0000'

In [45]:
base_tbstr = '<th scope="col" style="vertical-align: middle;">{}</th>'
tbstr = ['<tr>' for i in range(len(tmp_table))]
for i,v in enumerate(tmp_table):
    for j in range(8):
        if j == 0:
            tbstr[i] += base_tbstr.format(''.join(v[:2])+ '<br>'+ v[2])
        else:
            tbstr[i] += base_tbstr.format(v[j+2])
    tbstr[i] += '</tr>'
print(tbstr[0])

<tr><th scope="col" style="vertical-align: middle;">永和區福和段<br>0026-0000</th><th scope="col" style="vertical-align: middle;">否</th><th scope="col" style="vertical-align: middle;">否</th><th scope="col" style="vertical-align: middle;">否</th><th scope="col" style="vertical-align: middle;">否</th><th scope="col" style="vertical-align: middle;">否</th><th scope="col" style="vertical-align: middle;">否</th><th scope="col" style="vertical-align: middle;">否</th></tr>


'<tr><th scope="col" style="vertical-align: middle;">永和區福和段<br>0026-0000</th><th scope="col" style="vertical-align: middle;">否</th><th scope="col" style="vertical-align: middle;">否</th><th scope="col" style="vertical-align: middle;">否</th><th scope="col" style="vertical-align: middle;">否</th><th scope="col" style="vertical-align: middle;">否</th><th scope="col" style="vertical-align: middle;">否</th><th scope="col" style="vertical-align: middle;">否</th></tr><tr><th scope="col" style="vertical-align: middle;">永和區福和段<br>0027-0000</th><th scope="col" style="vertical-align: middle;">否</th><th scope="col" style="vertical-align: middle;">否</th><th scope="col" style="vertical-align: middle;">否</th><th scope="col" style="vertical-align: middle;">否</th><th scope="col" style="vertical-align: middle;">否</th><th scope="col" style="vertical-align: middle;">否</th><th scope="col" style="vertical-align: middle;">否</th></tr>'

In [51]:
    body = '''
    <div class="waterMark">
        <img style="position: fixed;bottom: 45%;left:28%;height:150; width:auto;" src="https://iot.thinktron.co/platform/ntpcagr/img/LOGO0.jpg" alt="NTPC" title="NTPC">
    </div>
    <p style="text-align:center;font-size:45px;font-family:Microsoft YaHei;">新北市政府農業局山坡地資訊查詢結果</p>
        <table id="searchResult-table" class="table table-sm table-bordered text-center" style="cursor: pointer;">
            <thead class="thead-dark">
                <tbody>
                <tr>
                    <th scope="col" style="width: 12.5%; vertical-align: middle;" >查詢時間：</th>
                    <th scope="col" colspan="4" style="width: 50%; vertical-align: middle !important;">{}</th>
                    <th scope="col" style="width: 12.5%; vertical-align: middle;" >查驗網址：</th>
                    <th scope="col" colspan="3";style="width: 37.5%; vertical-align: middle;"><img src="https://iot.thinktron.co/platform/ntpcagr/PDF/{}.jpg" alt="qr" title="qr"></th>
                </tr>
                <tr>
                    <th scope="col" style="width: 12.5%; vertical-align: middle;" >查詢地號</th>
                    <th scope="col" style="width: 12.5%; vertical-align: middle;" >是否座落<br>法定山坡地</th>
                    <th scope="col" style="width: 12.5%; vertical-align: middle;" >是否座落<br>特定水保區</th>
                    <th scope="col" style="width: 12.5%; vertical-align: middle;" >是否座落<br>國家公園</th>
                    <th scope="col" style="width: 12.5%; vertical-align: middle;" >是否座落<br>水庫集水區</th>
                    <th scope="col" style="width: 12.5%; vertical-align: middle;" >是否座落<br>保安林</th>
                    <th scope="col" style="width: 12.5%; vertical-align: middle;" >是否座落<br>山崩地滑<br>地質敏感區</th>
                    <th scope="col" style="width: 12.5%; vertical-align: middle;" >是否座落<br>土石流潛勢溪流<br>影響範圍</th>
                </tr>
                {}
                <tr>
                    <th scope="col" colspan="8"; style="text-align: left;">
                        <h5>
                        注意事項：<br>
                        1. 山坡地、特水區皆可作為本局正式證明文件之依據。<br>
                        2. 本頁查詢結果資料將保存一個月。<br>
                        3. 地籍資料仍以地政機關之資料為準，本系統查無相關地籍資料時，請逕洽本局查詢。<br>
                        4. 本系統之山坡地範圍不包含國有林事業區、試驗用林地及保安林地。<br>
                        5. 山坡地範圍及特定水土保持區倘因範圍有疑義者，請逕洽本局查詢。<br>
                        6. 國家公園、水庫集水區、保安林、地質敏感區及土石流潛勢溪流影響範圍由相關主管機關提供，不具證明效力。
                        </h5>
                    </th>
                </tr>
                </tbody>
            </thead>
        </table>
            '''.format('2021.10.13','QR',''.join(tbstr))
body

'\n<div class="waterMark">\n    <img style="position: fixed;bottom: 45%;left:28%;height:150; width:auto;" src="https://iot.thinktron.co/platform/ntpcagr/img/LOGO0.jpg" alt="NTPC" title="NTPC">\n</div>\n<p style="text-align:center;font-size:45px;font-family:Microsoft YaHei;">新北市政府農業局山坡地資訊查詢結果</p>\n    <table id="searchResult-table" class="table table-sm table-bordered text-center" style="cursor: pointer;">\n        <thead class="thead-dark">\n            <tbody>\n            <tr>\n                <th scope="col" style="width: 12.5%; vertical-align: middle;" >查詢時間：</th>\n                <th scope="col" colspan="4" style="width: 50%; vertical-align: middle !important;">2021.10.13</th>\n                <th scope="col" style="width: 12.5%; vertical-align: middle;" >查驗網址：</th>\n                <th scope="col" colspan="3";style="width: 37.5%; vertical-align: middle;"><img src="https://iot.thinktron.co/platform/ntpcagr/PDF/QR.jpg" alt="qr" title="qr"></th>\n            </tr>\n            <tr>\

In [52]:
def wkhtmltopdf(body, date,QR):
    head = '''
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" integrity="sha384-JcKb8q3iqJ61gNV9KGb8thSsNjpSL0n8PARn9HuZOnIxN0hoP+VmmDGMN5t9UJ0Z" crossorigin="anonymous">
    <link rel="stylesheet" href="https://use.fontawesome.com/releases/v5.8.2/css/all.css" integrity="sha384-oS3vJWv+0UjzBfQzYUhtDYW+Pj2yciDJxpsK1OYPAYjqT085Qq/1cq5FLXAZQ7Ay" crossorigin="anonymous">
    <link href="https://cdn.datatables.net/1.10.22/css/jquery.dataTables.min.css" rel="stylesheet" media="all">
    <link href="https://fonts.googleapis.com/css?family=Montserrat:400,700,200" rel="stylesheet" />
    <link href="https://maxcdn.bootstrapcdn.com/font-awesome/latest/css/font-awesome.min.css" rel="stylesheet">
    '''
    style = '''
    <style type="text/css">
        .waterMark {
            text-align:center;
            position:absolute;
            width:100%;
            height:100%;
            z-index:-9999;
            pointer-event:none;
            font-size:5em;
            font-weight:bold;
            font-family:Consolas;
            color:#EDEDED
        }
        .waterMark-inner{
            transform:rotate(-20deg);
            line-height:200px
        }
    </style>
    '''
    body = body
    options = {
        'encoding': 'utf-8',  #utf-8
        'page-height': '210',
        'page-width': '297',
        'background': ''
    }
    
    f = open(os.path.join(base_path,'PDF',QR+"_ziyu.html"),'w')
    f.write(head)
    f.write(style)
    f.write(body)
    
    f.close()
    pdfkit.from_file(os.path.join(base_path,'PDF',QR+"_ziyu.html"), os.path.join(base_path,'PDF',QR+"_ziyu.pdf"), configuration=config, options=options)

In [53]:
def export_single_cad(AA48_list,AA49_list, body):
    this_time = datetime.now().strftime("%Y/%m/%d %H:%M")
    qrstring = id_generator()
    generate_qrcode(qrstring)
    cad_list = []
    for AA48,AA49 in zip(AA48_list,AA49_list):
        cad = get_single_cad(AA48,AA49)
        cad_list.append(cad)
    wkhtmltopdf(body, this_time, qrstring)
    #wkhtmltopdf(this_time,qrstring,cad[0],cad[1],cad[2],cad[3],cad[4],cad[5],cad[6],cad[7],cad[8],cad[9])
AA48_list = ["1822", "1822"]
AA49_list = ['00260000', '00270000']
export_single_cad(AA48_list,AA49_list, body)

Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                      
